<a href="https://colab.research.google.com/github/Mikky-sout/MachineLearningOnly/blob/main/ProjectML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Heart Disease Prediction

**Import Librarys**

In [171]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn import metrics
import pandas_datareader.data as web
import pandas
from pandas_datareader import data as pdr
import datetime
import math
from sklearn import model_selection
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.over_sampling import RandomOverSampler 

**Prepare Dataset**

In [2]:
df = pd.read_csv("/content/drive/MyDrive/heart_2020_cleaned.csv")
df.head()
Y = pd.DataFrame(df['HeartDisease'])
df = df.drop(df[['HeartDisease']],axis=1)
Y

,HeartDisease
0,No
1,No
2,No
3,No
4,No
...,...
319790,Yes
319791,No
319792,No
319793,No


In [3]:
categorical_featureX = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df,columns=categorical_featureX,drop_first=True)
df.head()

,BMI,PhysicalHealth,MentalHealth,SleepTime,Smoking_Yes,AlcoholDrinking_Yes,Stroke_Yes,DiffWalking_Yes,Sex_Male,AgeCategory_25-29,...,Diabetic_Yes,Diabetic_Yes (during pregnancy),PhysicalActivity_Yes,GenHealth_Fair,GenHealth_Good,GenHealth_Poor,GenHealth_Very good,Asthma_Yes,KidneyDisease_Yes,SkinCancer_Yes
0,16.60,3.0,30.0,5.0,1,0,0,0,0,0,...,1,0,1,0,0,0,1,1,0,1
1,20.34,0.0,0.0,7.0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,26.58,20.0,30.0,8.0,1,0,0,0,1,0,...,1,0,1,1,0,0,0,1,0,0
3,24.21,0.0,0.0,6.0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,23.71,28.0,0.0,8.0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0


In [4]:
lb = LabelBinarizer()
labels = lb.fit_transform(Y)
labels = to_categorical(labels)
Y = np.array(labels)

In [5]:
Y = pd.DataFrame(Y)
Y.columns = ['HeartDisease_No','HeartDisease_Yes']
Y

,HeartDisease_No,HeartDisease_Yes
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
319790,0.0,1.0
319791,1.0,0.0
319792,1.0,0.0
319793,1.0,0.0


In [6]:
standard_scaler = preprocessing.StandardScaler();
X = pd.DataFrame(standard_scaler.fit_transform(df.values),index = df.index, columns=df.columns)
# Y = pd.DataFrame(standard_scaler.fit_transform(Y.values),index = X2.index, columns=X2.columns)


In [7]:
X.head()

,BMI,PhysicalHealth,MentalHealth,SleepTime,Smoking_Yes,AlcoholDrinking_Yes,Stroke_Yes,DiffWalking_Yes,Sex_Male,AgeCategory_25-29,...,Diabetic_Yes,Diabetic_Yes (during pregnancy),PhysicalActivity_Yes,GenHealth_Fair,GenHealth_Good,GenHealth_Poor,GenHealth_Very good,Asthma_Yes,KidneyDisease_Yes,SkinCancer_Yes
0,-1.844750,-0.046751,3.281069,-1.460354,1.193474,-0.27032,-0.198040,-0.401578,-0.951711,-0.236615,...,2.614905,-0.089814,0.538256,-0.348745,-0.640987,-0.191292,1.344886,2.541515,-0.195554,3.118419
1,-1.256338,-0.424070,-0.490039,-0.067601,-0.837890,-0.27032,5.049478,-0.401578,-0.951711,-0.236615,...,-0.382423,-0.089814,0.538256,-0.348745,-0.640987,-0.191292,1.344886,-0.393466,-0.195554,-0.320675
2,-0.274603,2.091388,3.281069,0.628776,1.193474,-0.27032,-0.198040,-0.401578,1.050739,-0.236615,...,2.614905,-0.089814,0.538256,2.867422,-0.640987,-0.191292,-0.743558,2.541515,-0.195554,-0.320675
3,-0.647473,-0.424070,-0.490039,-0.763977,-0.837890,-0.27032,-0.198040,-0.401578,-0.951711,-0.236615,...,-0.382423,-0.089814,-1.857852,-0.348745,1.560094,-0.191292,-0.743558,-0.393466,-0.195554,3.118419
4,-0.726138,3.097572,-0.490039,0.628776,-0.837890,-0.27032,-0.198040,2.490174,-0.951711,-0.236615,...,-0.382423,-0.089814,0.538256,-0.348745,-0.640987,-0.191292,1.344886,-0.393466,-0.195554,-0.320675


In [8]:
Y.head()

,HeartDisease_No,HeartDisease_Yes
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


**Under Sampling (For imbalance dataset)**

In [172]:
undersample = RandomUnderSampler(sampling_strategy='majority')
ros = RandomOverSampler(random_state=23)
X_us,Y_us = undersample.fit_resample(np.asarray(X), np.asarray(Y))
X_os,Y_os = ros.fit_resample(np.asarray(X), np.asarray(Y))

In [173]:
X_us = pd.DataFrame(X_us)
X_us.columns = X.columns
Y_us = pd.DataFrame(Y_us)

X_os = pd.DataFrame(X_os)
X_os.columns = X.columns
Y_os = pd.DataFrame(Y_os)


In [174]:
lb = LabelBinarizer()
labels = lb.fit_transform(Y_us)
labels = to_categorical(labels)
Y_us = np.array(labels)
Y_us = pd.DataFrame(Y_us)
Y_us.columns = ['HeartDisease_No','HeartDisease_Yes']


lb = LabelBinarizer()
labels = lb.fit_transform(Y_os)
labels = to_categorical(labels)
Y_os = np.array(labels)
Y_os = pd.DataFrame(Y_os)
Y_os.columns = ['HeartDisease_No','HeartDisease_Yes']

In [175]:
Y_us

,HeartDisease_No,HeartDisease_Yes
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
54741,0.0,1.0
54742,0.0,1.0
54743,0.0,1.0
54744,0.0,1.0


# Train Test Split

In [13]:
from sklearn.model_selection import train_test_split

In [177]:
x_train,x_test,y_train,y_test = train_test_split(X_us,Y_us,test_size=0.3,random_state=23)
x_train1,x_test1,y_train1,y_test1 = train_test_split(X,Y,test_size=0.3,random_state=23)
x_train2,x_test2,y_train2,y_test2 = train_test_split(X_os,Y_os,test_size=0.3,random_state=23)

# KNN Training

**Define parameter and Create Model**

In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [16]:
k = 51

modelKNN = KNeighborsClassifier(n_neighbors=k,p=2)
modelKNN.fit(x_train1,y_train1)

KNeighborsClassifier(n_neighbors=51)

In [17]:
pred = modelKNN.predict(x_test1)
KNNScore = accuracy_score(y_test1,pred)

In [18]:
print('KNN Accuracy :',KNNScore)

KNN Accuracy : 0.9167387610877745


In [19]:
df_pred = pd.DataFrame(pred)
y_pred = df_pred.idxmax(axis=1)

In [20]:
y_true = np.asarray(y_test1)
df_true = pd.DataFrame(y_true)
y_true = df_true.idxmax(axis=1)

In [21]:
print('--------- Confusion matrix-----------')
print(confusion_matrix(y_true, y_pred))
print('--------- Classification Report matrix -----------')
print(classification_report(y_true, y_pred))

--------- Confusion matrix-----------
[[87476   377]
 [ 7611   475]]
--------- Classification Report matrix -----------
              precision    recall  f1-score   support

           0       0.92      1.00      0.96     87853
           1       0.56      0.06      0.11      8086

    accuracy                           0.92     95939
   macro avg       0.74      0.53      0.53     95939
weighted avg       0.89      0.92      0.88     95939



# KNN Training With Oversampling

In [222]:
k = 51

modelKNN = KNeighborsClassifier(n_neighbors=k,p=2)
modelKNN.fit(x_train2,y_train2)

KNeighborsClassifier(n_neighbors=51)

In [223]:
pred_us = modelKNN.predict(x_test2)
KNNScore_us = accuracy_score(y_test2,pred_us)

In [224]:
print('KNN Accuracy :',KNNScore_us)

KNN Accuracy : 0.7785174461682264


In [225]:
df_pred_us = pd.DataFrame(pred_us)
y_pred_us = df_pred_us.idxmax(axis=1)

In [226]:
y_true_us = np.asarray(y_test2)
df_true_us = pd.DataFrame(y_true_us)
y_true_us = df_true_us.idxmax(axis=1)

In [227]:
print('--------- Confusion matrix-----------')
print(confusion_matrix(y_true_us, y_pred_us))
print('--------- Classification Report matrix -----------')
print(classification_report(y_true_us, y_pred_us))

--------- Confusion matrix-----------
[[63773 24034]
 [14826 72821]]
--------- Classification Report matrix -----------
              precision    recall  f1-score   support

           0       0.81      0.73      0.77     87807
           1       0.75      0.83      0.79     87647

    accuracy                           0.78    175454
   macro avg       0.78      0.78      0.78    175454
weighted avg       0.78      0.78      0.78    175454



# NeuralNetwork Training

In [127]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [214]:
modelNN = Sequential()
modelNN.add(Dense(128, input_dim=37, activation='relu'))
modelNN.add(Dense(128, activation='sigmoid'))
modelNN.add(Dense(128, activation='sigmoid'))
modelNN.add(Dropout(0.5))
modelNN.add(Dense(2, activation='softmax'))
modelNN.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [215]:
modelNN.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_95 (Dense)            (None, 128)               4864      
                                                                 
 dense_96 (Dense)            (None, 128)               16512     
                                                                 
 dense_97 (Dense)            (None, 128)               16512     
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_98 (Dense)            (None, 2)                 258       
                                                                 
Total params: 38,146
Trainable params: 38,146
Non-trainable params: 0
_________________________________________________________________


In [216]:
modelNN.fit(x_train2, y_train2, epochs=150, batch_size=128,verbose=1,validation_data=(x_test2,y_test2))

Epoch 1/150
3199/3199 [==============================] - 13s 4ms/step - loss: 0.4963 - accuracy: 0.7630 - val_loss: 0.4824 - val_accuracy: 0.7693
Epoch 2/150
3199/3199 [==============================] - 11s 3ms/step - loss: 0.4849 - accuracy: 0.7695 - val_loss: 0.4806 - val_accuracy: 0.7702
Epoch 3/150
3199/3199 [==============================] - 11s 3ms/step - loss: 0.4816 - accuracy: 0.7705 - val_loss: 0.4773 - val_accuracy: 0.7713
Epoch 4/150
3199/3199 [==============================] - 12s 4ms/step - loss: 0.4790 - accuracy: 0.7716 - val_loss: 0.4767 - val_accuracy: 0.7720
Epoch 5/150
3199/3199 [==============================] - 12s 4ms/step - loss: 0.4755 - accuracy: 0.7728 - val_loss: 0.4730 - val_accuracy: 0.7731
Epoch 6/150
3199/3199 [==============================] - 12s 4ms/step - loss: 0.4720 - accuracy: 0.7742 - val_loss: 0.4716 - val_accuracy: 0.7730
Epoch 7/150
3199/3199 [==============================] - 12s 4ms/step - loss: 0.4674 - accuracy: 0.7758 - val_loss: 0.4655 -

In [217]:
predNN = modelNN.predict(x_test2)


In [218]:
predNN = pd.DataFrame(predNN)
y_predNN = predNN.idxmax(axis=1)


y_trueNN = np.asarray(y_test2)
y_trueNN = pd.DataFrame(y_trueNN)
y_trueNN= y_trueNN.idxmax(axis=1)

In [219]:
NNScore = accuracy_score(y_trueNN,y_predNN)

In [220]:
print(NNScore)

0.8681135796277087


In [221]:
print('--------- Confusion matrix -> NN-----------')
print(confusion_matrix(y_trueNN,y_predNN))
print('--------- Classification Report matrix -> NN-----------')
print(classification_report(y_trueNN,y_predNN))

--------- Confusion matrix -> NN-----------
[[67050 20757]
 [ 2383 85264]]
--------- Classification Report matrix -> NN-----------
              precision    recall  f1-score   support

           0       0.97      0.76      0.85     87807
           1       0.80      0.97      0.88     87647

    accuracy                           0.87    175454
   macro avg       0.88      0.87      0.87    175454
weighted avg       0.89      0.87      0.87    175454

